notebook for eda2: https://github.com/fractaldatalearning/Capstone2/blob/main/notebooks/eda2_single_user.ipynb

In [5]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.lines as mlines

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

from sklearn.cluster import KMeans

from sklearn.tree import export_graphviz
from six import StringIO  
from IPython.display import Image  
import pydotplus

import itertools
from library.sb_utils import save_file

In [3]:
df = pd.read_csv('../data/processed/full_data_cleaned.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33819106 entries, 0 to 33819105
Data columns (total 15 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   order_by_user_sequence  int64  
 3   order_dow               int64  
 4   order_hour_of_day       int64  
 5   days_since_prior_order  float64
 6   product_id              int64  
 7   add_to_cart_sequence    int64  
 8   reordered               int64  
 9   product_name            object 
 10  aisle_name              object 
 11  dept_name               object 
 12  aisle_id                int64  
 13  department_id           int64  
 14  eval_set                object 
dtypes: float64(1), int64(10), object(4)
memory usage: 3.8+ GB


In [15]:
# Overall goal: look for key products or clusters of products

# Play around with a single aisle's products to see if 
# anything useful comes of it. Start with the milk aisle.

milk_aisle = df[df['aisle_name']=='milk']
milk_products = milk_aisle['product_name'].unique()
milk_aisle['product_name'].value_counts()

Organic Whole Milk                            142813
Organic Reduced Fat 2% Milk                    49374
2% Reduced Fat Milk                            38631
Whole Milk                                     36968
Organic Reduced Fat Milk                       36869
                                               ...  
Creamy Chocolate Milkshake                         3
Milk Magic Chocolate Milk Flavoring Straws         3
Torrential Tropical Punch                          3
Organic 1% Lowfat Chocolate Milk                   3
Enlighten Hemp Milk                                2
Name: product_name, Length: 243, dtype: int64

In [14]:
len(milk_products)

243

In [18]:
# Make it possible to model with the data

milk_aisle.columns

Index(['order_id', 'user_id', 'order_by_user_sequence', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order', 'product_id',
       'add_to_cart_sequence', 'reordered', 'product_name', 'aisle_name',
       'dept_name', 'aisle_id', 'department_id', 'eval_set'],
      dtype='object')

In [23]:
milk4models = milk_aisle.loc[:,['user_id', 'order_by_user_sequence', 
                                'order_dow','order_hour_of_day', 
                                'days_since_prior_order', 'product_id',
                                'add_to_cart_sequence', 'reordered', 
                                'aisle_id', 'department_id']]
milk4models.isnull().any()

user_id                   False
order_by_user_sequence    False
order_dow                 False
order_hour_of_day         False
days_since_prior_order     True
product_id                False
add_to_cart_sequence      False
reordered                 False
aisle_id                  False
department_id             False
dtype: bool

In [26]:
milk4models['days_since_prior_order'].sort_values().unique()

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29., 30., nan])

In [30]:
milk4models['days_since_prior_order'] = milk4models[
    'days_since_prior_order'].fillna(-1)
milk4models.isnull().any()

user_id                   False
order_by_user_sequence    False
order_dow                 False
order_hour_of_day         False
days_since_prior_order    False
product_id                False
add_to_cart_sequence      False
reordered                 False
aisle_id                  False
department_id             False
dtype: bool

In [31]:
model = KMeans(n_clusters=24)

# Fitting Model
model.fit(milk4models)

# Prediction on the entire data
all_predictions = model.predict(milk4models)

# Printing Predictions
print(all_predictions)

[ 3  3  3 ... 22 22 22]
